**Required Packages**:

In [ ]:
!pip install flair
!pip install vaderSentiment
import flair
import nltk
import textblob
import json
import spacy
import pandas as pd
import re
import html
from nltk.tokenize.punkt import TokenizerI
from textblob import TextBlob
from nltk import pos_tag
from flair.nn import Classifier
from flair.data import Sentence
from googleapiclient.discovery import build
from nltk.sentiment import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from google.colab import drive
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nltk.download('punkt')
nltk.download('wordnet')
!python -m textblob.download_corpora

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 52.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 38.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 54.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.9 MB/s eta 0:00:0

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


API Documentation Links for fast access:
1. https://developers.google.com/youtube/v3/docs/commentThreads/list
2. https://textblob.readthedocs.io/en/dev/
3. https://www.nltk.org/
4. https://flairnlp.github.io/docs/tutorial-basics/other-models
5. https://vadersentiment.readthedocs.io/en/latest/
6. https://spacy.io/api/doc/





Additionally, you can refer to these github repo (it has a youtube video): https://github.com/hellotinah/youtube_sentiment_analysis/

In [ ]:
drive.mount('/content/drive/')
pathToDatasets = '/content/drive/MyDrive/datasets/'

Mounted at /content/drive/


In [ ]:
class SentimentAnalysis():
    api = None
    def __init__(self):
        try:
            yt = build('youtube', 'v3', developerKey='')
            self.api = yt

        except Exception as e:
            print(f"Init error:{e}")

    def getCommentThread(self,
                        videoId='YbJOTdZBX1g',
                        layerZero='items',
                        layerOne='snippet',
                        layerTwo='topLevelComment',
                        maxResults=100,
                        textFormat='plainText',
                        order='relevance' ,
                        openMode='w+',
                        commentLimit=500
                        ):
        """Uses Google API client to retrieve YouTube's comment thread based on video ID and stores the data in a JSON object."""
        try:
            commentsList, authorNameList, likeCountList, datePostedList = [], [], [], []
            pageToken = None
            currentLimit = 0

            while True:
                commentThreadResponse = self.api.commentThreads().list(
                    videoId=videoId,
                    part=layerOne,
                    maxResults=maxResults,
                    textFormat=textFormat,
                    order=order,
                    pageToken=pageToken
                ).execute()

                for thread in commentThreadResponse['items']:
                    comment = thread[layerOne][layerTwo][layerOne]['textDisplay']
                    authorName = thread[layerOne][layerTwo][layerOne]['authorDisplayName']
                    likeCount = thread[layerOne][layerTwo][layerOne]['likeCount']
                    publishedAt = thread[layerOne][layerTwo][layerOne]['publishedAt']

                    commentsList.append(comment)
                    authorNameList.append(authorName)
                    likeCountList.append(likeCount)
                    datePostedList.append(publishedAt)

                    currentLimit += 1
                    if currentLimit >= commentLimit:
                        break

                if currentLimit >= commentLimit:
                    break

                if 'nextPageToken' in commentThreadResponse:
                    pageToken = commentThreadResponse['nextPageToken']  # Update the page token for the next iteration
                else:
                    break  # Break the loop if there are no more pages

            dataSet = {
                'comments': {i: commentsList[i] for i in range(len(commentsList))},
                'authorName': {i: authorNameList[i] for i in range(len(authorNameList))},
                'likeCount': {i: likeCountList[i] for i in range(len(likeCountList))},
                'datePosted': {i: datePostedList[i] for i in range(len(datePostedList))}
            }

            fileName = self.getVideoTitle(videoId) + '.json'

            with open(fileName, openMode) as file:
                json.dump(dataSet, file, indent=4)

            print(f"Data has been stored on '{fileName}'")
        except Exception as e:
            print(f"getCommentThread error: {e}")

    def getVideoTitle(self, videoId):
        """Returns a string  format of a youtube video title based on video ID."""
        try:
            videoResponse = self.api.videos().list(
                part='snippet',
                id=videoId
            ).execute()

            videoTitle = videoResponse['items'][0]['snippet']['title']

            return videoTitle
        except Exception as e:
            print(f"getVideoTitle error:{e} ")

    def readJsonData(self, currentDataSet='Harry & Meghan_ A New Life In Hollywood (FULL MOVIE).json', accessMode = 'r'):
        try:
            with open(pathToDatasets+currentDataSet, accessMode) as file:
                jsonDataSet = json.load(file)
            return jsonDataSet
        except Exception as e:
            print(f"readJsonData error: {e}")


PreProcessingTechniques Class for all implementations required for pre-processing data.

In [ ]:

class PreProcessingTechniques():
    #text blob
    #flair
    def __init__(self):
        pass

    def aggreggatedPreProcessing(self, inputFile):
        """Excludes Spelling correction, Lemmatization, and Tokenization (Since both of them return a list of individual tokens instead of sentences. So there's some ambiguity in how to utilize that.)"""
        aggregatedCleanedList = self.processStemming(
                                            self.processHtmlRemoval(
                                                self.processExtraSpacesRemoval(
                                                    self.processEmojiRemoval(
                                                        self.processNumberRemoval(
                                                            self.processLowerCase(inputFile))))))

        return aggregatedCleanedList

    def aggregatedPreProcessingV2(self, inputFile):
        """Excludes Spelling correction, Stemming, Lemmatization, and Tokenization (Since both of them return a list of individual tokens instead of sentences. So there's some ambiguity in how to utilize that.)"""
        aggregatedCleanedList = self.processHtmlRemoval(
                                    self.processExtraSpacesRemoval(
                                        self.processEmojiRemoval(
                                            self.processNumberRemoval(
                                                self.processLowerCase(inputFile)))))

        return aggregatedCleanedList

    def printer(self, inputFile):
        count=0
        for line in inputFile:
            print(f'{count}:{line}')
            count+=1

    def processEmojiRemoval(self, inputFile):
        processedList = []
        print("processEmojiRemoval output:")
        for line in inputFile:
            plainTextLine = html.unescape(line)
            processedComment = re.sub(r'[^a-zA-Z0-9\&\.\!\,\?\s]', '', plainTextLine)
            processedList.append(processedComment)
        return processedList

    def processExtraSpacesRemoval(self, inputFile):
        processedList = []
        print("processExtraSpacesRemoval output:")
        for line in inputFile:
            cleanedLine = re.sub(r'\s+',' ',line.strip())
            processedList.append(cleanedLine)
        return processedList

    def processHtmlRemoval(self, inputFile):
        processedList = []
        print("processHtmlRemoval output:")
        for line in inputFile:
            cleanedLine = re.sub(r'<[^>]+>',' ',line)
            processedList.append(cleanedLine)
        return processedList

    def processLemmatization(self, inputFile):
        processedList = []
        print("processLemmatization output:")
        nlp = spacy.load("en_core_web_sm")
        for line in inputFile:
            sentence = nlp(line)
            lemmatizedSentence = []
            for word in sentence:
                lemmatizedSentence.append(word.lemma_)
            processedList.append(lemmatizedSentence)
        return processedList

    def processLowerCase(self, inputFile):
            processedList = []
            print("processLowerCase output:")
            for line in inputFile:
                processedList.append(line.lower())
            return processedList

    def processNumberRemoval(self, inputFile):
        processedList = []
        print("processNumberRemoval output: ")
        for line in inputFile:
            cleanedLine = re.sub(r'\d+','', line)
            processedList.append(cleanedLine)
        return processedList

    def processSpellingCorrection(self, inputFile):
        count = 0
        processedList = []
        print("processSpellingCorrection output: ")
        for line in inputFile:
            correctedLine =  TextBlob(line)
            processedList.append(correctedLine.correct())
        return processedList

    def processPOS(self, inputFile):
        processedList = []
        print("processPOS output: ")
        for line in inputFile:
            blobLine = TextBlob(line)
            processedList.append(blobLine.tags)
        return processedList

    def processStemming(self, inputFile):
        processedList = []
        print("processStemming output:")
        stemm = PorterStemmer()
        for line in inputFile:
            stemmed_words = [stemm.stem(word) for word in line.split()]
            stemmedLine = ' '.join(stemmed_words)
            processedList.append(stemmedLine)
        return processedList

    def processTokenization(self, inputFile):
        processedList = []
        print("processTokenization output:")
        for line in inputFile:
            tokenizedLine = word_tokenize(line)
            processedList.append(tokenizedLine)
        return processedList

    def runSentimentAnalysis(self, inputFile, textBlobLib=True, flairLib=True, vaderSentimentLib=True, nltkLib=True):
        for line in inputFile:
            if textBlobLib:
                tbScore = TextBlob(line)
                print(f'TextBlob score: {tbScore.sentiment} - Comment: {line}')
            if flairLib:
                tagger = Classifier.load('sentiment')
                sentence = Sentence(line)
                tagger.predict(sentence)
                #Flair is 'state-of-the-art' so the model we're currently using is the original instead of the faster model for better accuracy.
                print(f'Flair score: {sentence}')
            if vaderSentimentLib:
                analyzer = SentimentIntensityAnalyzer()
                vaderScore = analyzer.polarity_scores(line)
                vsScore = vaderScore['compound']
                print(f'Vader score: {vsScore} - Comment: {line}')
            if nltkLib:
                sia = SIA()
                sentimentScore = sia.polarity_scores(line)
                compoundedScore = sentimentScore['compound']
                print(f'NLTK: {compoundedScore} - Comment: {line}')

ppt = PreProcessingTechniques()


In [ ]:
SA2 = SentimentAnalysis()
#stores json into dataSet variable
dataSet = SA2.readJsonData()
#gets only 21 'comments'
commentList = []
for comment in range(21):
        commentList.append(dataSet['comments'][str(comment+122)])
        #122 is merely a randomized number that actually provided a good set of 21 sentences that are apt for preprocessing techniques to be cleansed.

for index in range(len(commentList)):
    print(f"{index}:{commentList[index]}")

#we'll use this as the input

0:Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
1:Harry USED TO BE so adorable. What happened to him? he changed since 2018.
2:What is royal. We are human beings
3:We love king Harry <a href="https://www.youtube.com/watch?v=2b9uOhf1L4s&amp;t=35m48s">35:48</a>
4:I too have a different color of skin it is white not just any white but Canadian white I think I have suffered here in Britain due to my extreme whiteness and am seriously considering writing a book about the racism I have been subjected to due to my extreme whiteness. Perhaps Meghan could help me. LOL
5:I love both of them❤
6:Even a farther doesn&#39;t think. Disappointing. But it is reality
7:Megans comment &quot;We are one plane crash from the throne&quot;.  In her mind she will become queen.  Craziest Woman Ever!!!
8:Billybord and sinus wave ❤❤❤
9:Look what happened to Princess Diana.
10:They are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:I LOVE Harry and Meghan 💙❤️🙏
12:Be the true hea

In [ ]:
ppt.printer(ppt.processLowerCase(commentList))

processLowerCase output:
0:edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
1:harry used to be so adorable. what happened to him? he changed since 2018.
2:what is royal. we are human beings
3:we love king harry <a href="https://www.youtube.com/watch?v=2b9uohf1l4s&amp;t=35m48s">35:48</a>
4:i too have a different color of skin it is white not just any white but canadian white i think i have suffered here in britain due to my extreme whiteness and am seriously considering writing a book about the racism i have been subjected to due to my extreme whiteness. perhaps meghan could help me. lol
5:i love both of them❤
6:even a farther doesn&#39;t think. disappointing. but it is reality
7:megans comment &quot;we are one plane crash from the throne&quot;.  in her mind she will become queen.  craziest woman ever!!!
8:billybord and sinus wave ❤❤❤
9:look what happened to princess diana.
10:they are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:i love harry and megha

In [ ]:
ppt.printer(ppt.processEmojiRemoval(commentList))

processEmojiRemoval output:
0:Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
1:Harry USED TO BE so adorable. What happened to him? he changed since 2018.
2:What is royal. We are human beings
3:We love king Harry a hrefhttpswww.youtube.comwatch?v2b9uOhf1L4s&t35m48s3548a
4:I too have a different color of skin it is white not just any white but Canadian white I think I have suffered here in Britain due to my extreme whiteness and am seriously considering writing a book about the racism I have been subjected to due to my extreme whiteness. Perhaps Meghan could help me. LOL
5:I love both of them
6:Even a farther doesnt think. Disappointing. But it is reality
7:Megans comment We are one plane crash from the throne.  In her mind she will become queen.  Craziest Woman Ever!!!
8:Billybord and sinus wave 
9:Look what happened to Princess Diana.
10:They are perfect 
11:I LOVE Harry and Meghan 
12:Be the true heart of the people they love you stan

In [ ]:
ppt.printer(ppt.processExtraSpacesRemoval(commentList))

processExtraSpacesRemoval output:
0:Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
1:Harry USED TO BE so adorable. What happened to him? he changed since 2018.
2:What is royal. We are human beings
3:We love king Harry <a href="https://www.youtube.com/watch?v=2b9uOhf1L4s&amp;t=35m48s">35:48</a>
4:I too have a different color of skin it is white not just any white but Canadian white I think I have suffered here in Britain due to my extreme whiteness and am seriously considering writing a book about the racism I have been subjected to due to my extreme whiteness. Perhaps Meghan could help me. LOL
5:I love both of them❤
6:Even a farther doesn&#39;t think. Disappointing. But it is reality
7:Megans comment &quot;We are one plane crash from the throne&quot;. In her mind she will become queen. Craziest Woman Ever!!!
8:Billybord and sinus wave ❤❤❤
9:Look what happened to Princess Diana.
10:They are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:I LOVE Harry an

In [ ]:
ppt.printer(ppt.processHtmlRemoval(commentList))

processHtmlRemoval output:
0:Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
1:Harry USED TO BE so adorable. What happened to him? he changed since 2018.
2:What is royal. We are human beings
3:We love king Harry  35:48 
4:I too have a different color of skin it is white not just any white but Canadian white I think I have suffered here in Britain due to my extreme whiteness and am seriously considering writing a book about the racism I have been subjected to due to my extreme whiteness. Perhaps Meghan could help me. LOL
5:I love both of them❤
6:Even a farther doesn&#39;t think. Disappointing. But it is reality
7:Megans comment &quot;We are one plane crash from the throne&quot;.  In her mind she will become queen.  Craziest Woman Ever!!!
8:Billybord and sinus wave ❤❤❤
9:Look what happened to Princess Diana.
10:They are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:I LOVE Harry and Meghan 💙❤️🙏
12:Be the true heart of the people they love you stand tall 

In [ ]:
ppt.printer(ppt.processTokenization(commentList))

processTokenization output:
0:['Edits', 'are', 'continuously', 'repeated', 'only', 'what', 'we', 'know', 'already', 'was', 'hoping', 'it', 'would', 'be', 'a', 'fresh', 'Outlook', '.']
1:['Harry', 'USED', 'TO', 'BE', 'so', 'adorable', '.', 'What', 'happened', 'to', 'him', '?', 'he', 'changed', 'since', '2018', '.']
2:['What', 'is', 'royal', '.', 'We', 'are', 'human', 'beings']
3:['We', 'love', 'king', 'Harry', '<', 'a', 'href=', "''", 'https', ':', '//www.youtube.com/watch', '?', 'v=2b9uOhf1L4s', '&', 'amp', ';', 't=35m48s', "''", '>', '35:48', '<', '/a', '>']
4:['I', 'too', 'have', 'a', 'different', 'color', 'of', 'skin', 'it', 'is', 'white', 'not', 'just', 'any', 'white', 'but', 'Canadian', 'white', 'I', 'think', 'I', 'have', 'suffered', 'here', 'in', 'Britain', 'due', 'to', 'my', 'extreme', 'whiteness', 'and', 'am', 'seriously', 'considering', 'writing', 'a', 'book', 'about', 'the', 'racism', 'I', 'have', 'been', 'subjected', 'to', 'due', 'to', 'my', 'extreme', 'whiteness', '.', 'Per

In [ ]:
ppt.printer(ppt.processLemmatization(commentList))

processLemmatization output:
0:['edit', 'be', 'continuously', 'repeat', 'only', 'what', 'we', 'know', 'already', 'be', 'hope', 'it', 'would', 'be', 'a', 'fresh', 'Outlook', '.']
1:['Harry', 'use', 'to', 'be', 'so', 'adorable', '.', 'what', 'happen', 'to', 'he', '?', 'he', 'change', 'since', '2018', '.']
2:['what', 'be', 'royal', '.', 'we', 'be', 'human', 'being']
3:['we', 'love', 'king', 'Harry', '<', 'a', 'href="https://www.youtube.com', '/', 'watch?v=2b9uohf1l4s&amp;t=35m48s">35:48</a', '>']
4:['I', 'too', 'have', 'a', 'different', 'color', 'of', 'skin', 'it', 'be', 'white', 'not', 'just', 'any', 'white', 'but', 'canadian', 'white', 'I', 'think', 'I', 'have', 'suffer', 'here', 'in', 'Britain', 'due', 'to', 'my', 'extreme', 'whiteness', 'and', 'be', 'seriously', 'consider', 'write', 'a', 'book', 'about', 'the', 'racism', 'I', 'have', 'be', 'subject', 'to', 'due', 'to', 'my', 'extreme', 'whiteness', '.', 'perhaps', 'Meghan', 'could', 'help', 'I', '.', 'LOL']
5:['I', 'love', 'both', 'of

In [ ]:
ppt.printer(ppt.processStemming(commentList))

processStemming output:
0:edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
1:harri use to be so adorable. what happen to him? he chang sinc 2018.
2:what is royal. we are human be
3:we love king harri <a href="https://www.youtube.com/watch?v=2b9uohf1l4s&amp;t=35m48s">35:48</a>
4:i too have a differ color of skin it is white not just ani white but canadian white i think i have suffer here in britain due to my extrem white and am serious consid write a book about the racism i have been subject to due to my extrem whiteness. perhap meghan could help me. lol
5:i love both of them❤
6:even a farther doesn&#39;t think. disappointing. but it is realiti
7:megan comment &quot;w are one plane crash from the throne&quot;. in her mind she will becom queen. craziest woman ever!!!
8:billybord and sinu wave ❤❤❤
9:look what happen to princess diana.
10:they are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:i love harri and meghan 💙❤️🙏
12:be the true heart of the peopl they love yo

In [ ]:
ppt.printer(ppt.processSpellingCorrection(commentList))

processSpellingCorrection output: 
0:Its are continuously repeated only what we know already was hoping it would be a fresh Outlook.
1:Carry USED of of so adorable. That happened to him? he changed since 2018.
2:That is royal. He are human beings
3:He love king Carry <a he="http://www.couture.com/watch?v=2b9uOhf1L4s&amp;t=35m48s">35:48</a>
4:I too have a different color of skin it is white not just any white but Canadian white I think I have suffered here in Britain due to my extreme whiteness and am seriously considering writing a book about the racism I have been subjected to due to my extreme whiteness. Perhaps Began could help me. LOL
5:I love both of them❤
6:Even a farther doesn&#39;t think. Disappointing. But it is reality
7:Began comment &quit;He are one plane crash from the throne&quit;.  In her mind she will become queen.  Craziest Woman Over!!!
8:Billybord and sinus wave ❤❤❤
9:Took what happened to Princess Diana.
10:They are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:I LOVE Carry and Be

In [ ]:
ppt.printer(ppt.processPOS(commentList))

processPOS output: 
0:[('Edits', 'NNS'), ('are', 'VBP'), ('continuously', 'RB'), ('repeated', 'VBN'), ('only', 'RB'), ('what', 'WP'), ('we', 'PRP'), ('know', 'VBP'), ('already', 'RB'), ('was', 'VBD'), ('hoping', 'VBG'), ('it', 'PRP'), ('would', 'MD'), ('be', 'VB'), ('a', 'DT'), ('fresh', 'JJ'), ('Outlook', 'NNP')]
1:[('Harry', 'NNP'), ('USED', 'NNP'), ('TO', 'NNP'), ('BE', 'NNP'), ('so', 'RB'), ('adorable', 'JJ'), ('What', 'WP'), ('happened', 'VBD'), ('to', 'TO'), ('him', 'PRP'), ('he', 'PRP'), ('changed', 'VBD'), ('since', 'IN'), ('2018', 'CD')]
2:[('What', 'WP'), ('is', 'VBZ'), ('royal', 'NN'), ('We', 'PRP'), ('are', 'VBP'), ('human', 'JJ'), ('beings', 'NNS')]
3:[('We', 'PRP'), ('love', 'VBP'), ('king', 'VBG'), ('Harry', 'NNP'), ('<', 'VBD'), ('a', 'DT'), ('href=', 'NN'), ('https', 'NN'), ('//www.youtube.com/watch', 'NN'), ('v=2b9uOhf1L4s', 'NN'), ('&', 'CC'), ('amp', 'NN'), ('t=35m48s', 'CC'), ('>', 'VBZ'), ('35:48', 'CD'), ('<', 'NN'), ('/a', 'NN'), ('>', 'NN')]
4:[('I', 'PRP'), ('

In [ ]:
ppt.printer(ppt.processNumberRemoval(commentList))

processNumberRemoval output: 
0:Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
1:Harry USED TO BE so adorable. What happened to him? he changed since .
2:What is royal. We are human beings
3:We love king Harry <a href="https://www.youtube.com/watch?v=buOhfLs&amp;t=ms">:</a>
4:I too have a different color of skin it is white not just any white but Canadian white I think I have suffered here in Britain due to my extreme whiteness and am seriously considering writing a book about the racism I have been subjected to due to my extreme whiteness. Perhaps Meghan could help me. LOL
5:I love both of them❤
6:Even a farther doesn&#;t think. Disappointing. But it is reality
7:Megans comment &quot;We are one plane crash from the throne&quot;.  In her mind she will become queen.  Craziest Woman Ever!!!
8:Billybord and sinus wave ❤❤❤
9:Look what happened to Princess Diana.
10:They are perfect ♥️♥️♥️♥️♥️♥️♥️♥️♥️🥰
11:I LOVE Harry and Meghan 💙❤️🙏
12:Be 

In [ ]:
#Without preprocessing techniques
ppt.runSentimentAnalysis(commentList)

TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
2023-06-15 12:22:58,049 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmphi1q2_16


100%|██████████| 253M/253M [00:28<00:00, 9.36MB/s]

2023-06-15 12:23:26,992 copying /tmp/tmphi1q2_16 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-06-15 12:23:27,555 removing temp file /tmp/tmphi1q2_16


Flair score: Sentence[18]: "Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
NLTK: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
Flair score: Sentence[17]: "Harry USED TO BE so adorable. What happened to him? he changed since 2018." → NEGATIVE (0.9968)
Vader score: 0.6269 - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
NLTK: 0.6269 - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
TextBlob score: Sentiment(polarity=0.0, subjectivity=0.1) - Comment: What is royal. We are human beings
Flair score: Sentence[8]:

In [ ]:
ppt.runSentimentAnalysis(ppt.processEmojiRemoval(commentList))

processEmojiRemoval output:
TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
Flair score: Sentence[18]: "Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
NLTK: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
Flair score: Sentence[17]: "Harry USED TO BE so adorable. What happened to him? he changed since 2018." → NEGATIVE (0.9968)
Vader score: 0.6269 - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
NLTK: 0.6269 - Comment: Harry U

In [ ]:
ppt.runSentimentAnalysis(ppt.processExtraSpacesRemoval(commentList))

processExtraSpacesRemoval output:
TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
Flair score: Sentence[18]: "Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
NLTK: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
Flair score: Sentence[17]: "Harry USED TO BE so adorable. What happened to him? he changed since 2018." → NEGATIVE (0.9968)
Vader score: 0.6269 - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
NLTK: 0.6269 - Comment: H

In [ ]:
ppt.runSentimentAnalysis(ppt.processHtmlRemoval(commentList))

processHtmlRemoval output:
TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
Flair score: Sentence[18]: "Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
NLTK: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
Flair score: Sentence[17]: "Harry USED TO BE so adorable. What happened to him? he changed since 2018." → NEGATIVE (0.9968)
Vader score: 0.6269 - Comment: Harry USED TO BE so adorable. What happened to him? he changed since 2018.
NLTK: 0.6269 - Comment: Harry US

In [ ]:
ppt.runSentimentAnalysis(ppt.processLowerCase(commentList))

processLowerCase output:
TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
Flair score: Sentence[18]: "edits are continuously repeated only what we know already was hoping it would be a fresh outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
NLTK: 0.6249 - Comment: edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: harry used to be so adorable. what happened to him? he changed since 2018.
Flair score: Sentence[17]: "harry used to be so adorable. what happened to him? he changed since 2018." → NEGATIVE (0.9968)
Vader score: 0.6269 - Comment: harry used to be so adorable. what happened to him? he changed since 2018.
NLTK: 0.6269 - Comment: harry used

In [ ]:
ppt.runSentimentAnalysis(ppt.processNumberRemoval(commentList))

processNumberRemoval output: 
TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
Flair score: Sentence[18]: "Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
NLTK: 0.6249 - Comment: Edits are continuously repeated only what we know already was hoping it would be a fresh Outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: Harry USED TO BE so adorable. What happened to him? he changed since .
Flair score: Sentence[16]: "Harry USED TO BE so adorable. What happened to him? he changed since ." → NEGATIVE (0.9891)
Vader score: 0.6269 - Comment: Harry USED TO BE so adorable. What happened to him? he changed since .
NLTK: 0.6269 - Comment: Harry USED TO BE 

In [ ]:
ppt.runSentimentAnalysis(ppt.processStemming(commentList))

processStemming output:
TextBlob score: Sentiment(polarity=0.3, subjectivity=0.5) - Comment: edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
Flair score: Sentence[18]: "edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook." → POSITIVE (0.7244)
Vader score: 0.6369 - Comment: edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
NLTK: 0.6369 - Comment: edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: harri use to be so adorable. what happen to him? he chang sinc 2018.
Flair score: Sentence[17]: "harri use to be so adorable. what happen to him? he chang sinc 2018." → NEGATIVE (0.9991)
Vader score: 0.6269 - Comment: harri use to be so adorable. what happen to him? he chang sinc 2018.
NLTK: 0.6269 - Comment: harri use to be so adorable. what happen to him? he chang sinc 2018.
TextBl

In [ ]:
ppt.printer(ppt.aggreggatedPreProcessing(commentList))

processLowerCase output:
processNumberRemoval output: 
processEmojiRemoval output:
processExtraSpacesRemoval output:
processHtmlRemoval output:
processStemming output:
0:edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
1:harri use to be so adorable. what happen to him? he chang sinc .
2:what is royal. we are human be
3:we love king harri a hrefhttpswww.youtube.comwatch?vbuohfls&tmsa
4:i too have a differ color of skin it is white not just ani white but canadian white i think i have suffer here in britain due to my extrem white and am serious consid write a book about the racism i have been subject to due to my extrem whiteness. perhap meghan could help me. lol
5:i love both of them
6:even a farther doesn&t think. disappointing. but it is realiti
7:megan comment we are one plane crash from the throne. in her mind she will becom queen. craziest woman ever!!!
8:billybord and sinu wave
9:look what happen to princess diana.
10:they are perfect
11:i love

In [ ]:
#Aggregated list for score.
ppt.runSentimentAnalysis(ppt.aggreggatedPreProcessing(commentList))

processLowerCase output:
processNumberRemoval output: 
processEmojiRemoval output:
processExtraSpacesRemoval output:
processHtmlRemoval output:
processStemming output:
TextBlob score: Sentiment(polarity=0.3, subjectivity=0.5) - Comment: edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
Flair score: Sentence[18]: "edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook." → POSITIVE (0.7244)
Vader score: 0.6369 - Comment: edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
NLTK: 0.6369 - Comment: edit are continu repeat onli what we know alreadi wa hope it would be a fresh outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: harri use to be so adorable. what happen to him? he chang sinc .
Flair score: Sentence[16]: "harri use to be so adorable. what happen to him? he chang sinc ." → NEGATIVE (0.9993)
Vader score: 0.6269 - Comment: harri use to be so adorable. wha

In [ ]:
ppt.printer(ppt.aggregatedPreProcessingV2(commentList))

processLowerCase output:
processNumberRemoval output: 
processEmojiRemoval output:
processExtraSpacesRemoval output:
processHtmlRemoval output:
0:edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
1:harry used to be so adorable. what happened to him? he changed since .
2:what is royal. we are human beings
3:we love king harry a hrefhttpswww.youtube.comwatch?vbuohfls&tmsa
4:i too have a different color of skin it is white not just any white but canadian white i think i have suffered here in britain due to my extreme whiteness and am seriously considering writing a book about the racism i have been subjected to due to my extreme whiteness. perhaps meghan could help me. lol
5:i love both of them
6:even a farther doesn&t think. disappointing. but it is reality
7:megans comment we are one plane crash from the throne. in her mind she will become queen. craziest woman ever!!!
8:billybord and sinus wave
9:look what happened to princess diana.
10:t

In [ ]:
#Aggregated list for score.
ppt.runSentimentAnalysis(ppt.aggregatedPreProcessingV2(commentList))

processLowerCase output:
processNumberRemoval output: 
processEmojiRemoval output:
processExtraSpacesRemoval output:
processHtmlRemoval output:
TextBlob score: Sentiment(polarity=0.15, subjectivity=0.75) - Comment: edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
Flair score: Sentence[18]: "edits are continuously repeated only what we know already was hoping it would be a fresh outlook." → NEGATIVE (0.9952)
Vader score: 0.6249 - Comment: edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
NLTK: 0.6249 - Comment: edits are continuously repeated only what we know already was hoping it would be a fresh outlook.
TextBlob score: Sentiment(polarity=0.5, subjectivity=1.0) - Comment: harry used to be so adorable. what happened to him? he changed since .
Flair score: Sentence[16]: "harry used to be so adorable. what happened to him? he changed since ." → NEGATIVE (0.9891)
Vader score: 0.6269 - Comment